This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules

import requests
import json



Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [43]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/'
params = {'api_key' : API_KEY, 'start_date' : '2020-11-10', 'end_date': '2020-11-11'}

r = requests.get(url, params = params)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.url)
r.status_code
print(r.content)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [79]:
params_2017 = {'api_key' : API_KEY, 'start_date' : '2017-01-01', 'end_date': '2017-12-31'}

r2017 = requests.get(url, params = params_2017)
r2017.url

'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/?api_key=nxz75u8dqzy7ouuNkqe_&start_date=2017-01-01&end_date=2017-12-31'

 2. Convert the returned JSON object into a Python dictionary.

In [81]:
afx = json.loads(r2017.text)

In [ ]:
print(afx.keys())

for p_id, p_info in afx.items():
    print("\nPerson ID:", p_id)
    
    for key in p_info:
        print(key + ':', p_info[key])

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [ ]:
data = afx['dataset']['data']
opens = [sublist[1:2] for sublist in data]
opens_flat = [x for xs in opens for x in xs]
opens_filt = list(filter(None, opens_flat))
#equivalent to 
#flat_list = []
#for xs in highs:
#    for x in xs:
#        flat_list.append(x)

print("The max of the open price during this time period : ", max(opens_filt))
print("The min of the open price during this time period : ", min(opens_filt))


4. What was the largest change in any one day (based on High and Low price)?

In [102]:
hi = [sublist[2:3] for sublist in data]
lo = [sublist[3:4] for sublist in data]
hi_flat = [x for xs in hi for x in xs]
lo_flat = [x for xs in lo for x in xs]

In [103]:
max_chg = 0
for x in range(0,len(hilo)):
    new_chg = round(hi_flat[x] - lo_flat[x], 2)
    if new_chg > max_chg:
        max_chg = new_chg

print("The largest change in any one day : ",max_chg)

The largest change in any one day :  2.81


5. What was the largest change between any two days (based on Closing Price)?

In [105]:
close = [sublist[4:5] for sublist in data]
close_flat = [x for xs in close for x in xs]

In [107]:
max_chg = 0
for x in range(0,len(close)-1):
    new_chg = round(close_flat[x+1] - close_flat[x], 2)
    if new_chg > max_chg:
        max_chg = new_chg

print("The largest change in any one day : ",max_chg)

The largest change in any one day :  2.56


6. What was the average daily trading volume during this year?

In [114]:
traded = [sublist[6:7] for sublist in data]
traded_flat = [x for xs in traded for x in xs]

In [115]:
print("The average traded volume during this time period : ", round((sum(traded_flat)/len(traded_flat)),2))

The average traded volume during this time period :  89124.34
